# Proper tool use in Cohere

Learning goal: Understand the mechanism provided by cohere to officially support 
tool use in LLMs. 

## Short description

- We will use a LLM to answer questions normally, except that we define a "tool" that performs some functionality (in this case, getting the exact calories of a food item).
- Cohere offers a mechanism so that this tool is registered -- in which case 
the LLM can "pay attention" to whatever question that seems to need the tool and
alerts us. 
- In such a case, we shoud run the tool incorporate the result in the chat history, 
and query the LLM again.

## Environment preparation

First, load your API keys and initialize the API manager objects. We will use `cohere`as LLM and will use the Open Food Facts API to query information about calories of food items.


In [12]:
import cohere
import json
import openfoodfacts

with open("cohere.key") as f:
    COHERE_API_KEY = f.read()
cohere_client = cohere.ClientV2(COHERE_API_KEY)
api = openfoodfacts.API(user_agent="MyAwesomeApp/1.0")


## Defining the tool

Making cohere aware of the tool requires two steps. The first is defining what the tool
does as a python function. 

Here is a function that gets the name of a food item, queries the Open Food Facts database, 
and returns the number of calories of the item. 

In [13]:
def get_calories(name):
    """
    Get the calories of a product by its name.
    """
    result = api.product.text_search(name)

    # This just takes the first item of the search. 
    # In a real application, you would need to do more error checking.
    first_match = result['products'][0]
    id = first_match['_id']
    result = api.product.get(id, fields=["code", "product_name", "energy-kcal_100g"])
    return result['energy-kcal_100g']

The second step is to write a dictionary in a specific format. 

That object 
- Defines the name of the function.
- Describes what the tool does. 
- Defines the arguments for the tool. 


In [14]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_calories",
            "description": "Gets the exact calories of a given food item",
            "parameters": {
                "type": "object",
                "properties": {
                    "product": {
                        "type": "string",
                        "description": "the name of the food item, e.g. Donettes",
                    }
                },
                "required": ["product"],
            },
        },
    },
]


## Using the tool in the LLM

### a) Making the call

In order to make the LLM aware of the tool, we just add the argument
`tools` to the call. 

In [25]:
user_input = input()

messages = [
    {'role': 'user', 'content': user_input}
]

response = cohere_client.chat(
    model="command-a-03-2025", messages=messages, tools=tools
)



What occurs exactly when we use this? The LLM *internally* evaluates if the question is
related to the functionality of the tool. It does so by **comparing the question with 
the description of the tool**. 

Note that, if the question is unrelated to calories of food items, the LLM will respond normally. But if the question is related to calories, note the response of the LLM outputs the so-called "tool plan".

The tool plan includes a text message with an intention of what the LLM plans to do, but also the specific information (name of function and arguments) that it will execute. 

In [26]:
print(response.message.json(indent=2))

{
  "role": "assistant",
  "tool_calls": [
    {
      "id": "get_calories_egmjf4txxrgp",
      "type": "function",
      "function": {
        "name": "get_calories",
        "arguments": "{\"product\":\"Mikado\"}"
      }
    }
  ],
  "tool_plan": "I will look up the number of calories in a bag of Mikado."
}


Before continuing, let's keep track of all these messages, as we will need them later. 

In [17]:
if response.message.tool_calls:
    messages.append(
        {
            "role": "assistant",
            "tool_plan": response.message.tool_plan,
            "tool_calls": response.message.tool_calls,
        }
    )

### b) Executing the tool

The LLM software can't execute the tool, this is the responsibility of the 
programmer. 

However, the response object has the information structured in a way that already systematizes the tool call. 

Here is an example for our tool. Note if you had more than one tool, the code for each tool would be repetitive and you can even factor it easily. 

In [30]:
if response.message.tool_calls:
    for tc in response.message.tool_calls:

        if tc.function.name == 'get_calories':
            dict_string = tc.function.arguments
            arg_dict = json.loads(dict_string)
            calories = get_calories(arg_dict["product"])

            calories_message = f"""
            According to the OpenFoodFacts database, the calories of {arg_dict["product"]}
            are {calories}
            """

            messages.append({'role': 'tool', 'tool_call_id': tc.id, 'content': calories_message})
        else:
            raise ValueError("Unknown tool")



Again, we collect the messages and store them in the `messages` list, which now contains everything we've added: 

- The original message
- The response by the LLM with the tool plan
- The output of the tool call

In [31]:
messages

[{'role': 'user', 'content': 'Number of calories in a bag of Mikado'},
 {'role': 'tool',
  'tool_call_id': 'get_calories_egmjf4txxrgp',
  'content': '\n            According to the OpenFoodFacts database, the calories of Mikado\n            are 477\n            '},
 {'role': 'tool',
  'tool_call_id': 'get_calories_egmjf4txxrgp',
  'content': '\n            According to the OpenFoodFacts database, the calories of Mikado\n            are 477\n            '}]

### c) Making the final call

With that information, we can now make the final call to the LLM. 
When seeing all these messages, the LLM will compile the final answer from the original question and the tool response. 

In [32]:
response = cohere_client.chat(
    messages=messages,
    model="command-r-plus"
)



BadRequestError: status_code: 400, body: {'message': "invalid tool message at messages[1]: tool call id 'get_calories_egmjf4txxrgp' not found in previous tool calls"}

The nice thing of the tool is that the output object contains the "grounding" information: 

In [29]:
print(response.json(indent=3))

{
   "id": "fea0d8dd-ea6c-4577-bdc1-3d9ca5f56f8c",
   "finish_reason": "TOOL_CALL",
   "message": {
      "role": "assistant",
      "tool_calls": [
         {
            "id": "get_calories_egmjf4txxrgp",
            "type": "function",
            "function": {
               "name": "get_calories",
               "arguments": "{\"product\":\"Mikado\"}"
            }
         }
      ],
      "tool_plan": "I will look up the number of calories in a bag of Mikado."
   },
   "usage": {
      "billed_units": {
         "input_tokens": 37.0,
         "output_tokens": 25.0
      },
      "tokens": {
         "input_tokens": 1459.0,
         "output_tokens": 54.0
      }
   }
}
